### Datasets
http://www.manythings.org/anki/  (Download and unzip mar-eng.zip file)

In [1]:
from google.colab import files
import io

uploaded = files.upload()


Saving francais_fongbe_revu_sans_doublons2.txt to francais_fongbe_revu_sans_doublons2.txt


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pickle
import io

#print(type (uploaded['francais_fongbe_revu_sans_doublons2.txt']))

In [0]:

lines = pd.read_csv(io.StringIO(uploaded["francais_fongbe_revu_sans_doublons2.txt"].decode("utf-8")), names=['eng', 'mar'], encoding="utf-8", engine="python", delimiter=" : ")



In [0]:
lines = lines[:35000];


In [12]:
lines.shape

(35000, 2)

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
#lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [19]:
lines.sample(10)

,eng,mar
24503,il a détruit complètement la case,START_ e kíjá xɔ ɔ́ _END
27415,j’ai une plaie inguérissable,START_ un ɖò cákpásúahlo jɛ wɛ̀ _END
23864,il m’a heurté violemment,START_ é xò aglo mì gblawùn _END
14348,il y a eu la famine,START_ adɔ tɔ́n _END
6772,au niveau de,START_ lixò _END
2823,tenue,START_ hăɖotè _END
1467,glace,START_ glăsì _END
33014,et ses chemises sont de plus en plus laides,START_ bo awu tɔn lε ɖo nyla kan de ji wε _END
19271,tu travailles bien,START_ a nɔ w’àzɔ̆ ganjí _END
28579,les feuilles se détachent et senvolent,START_ folo lε ɖo ɖiɖe wε _END


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [21]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

20

In [22]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

21

In [23]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(12210, 13211)

In [24]:
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens += 1 # Par francis pour eviter invalidArgumentError
num_decoder_tokens, num_encoder_tokens

(13212, 12211)

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [27]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
11652,histoire vraie,START_ tankpikpi _END
56,ver,START_ wĕ _END
34199,linstituteur informé du vol ne dit rien dabord,START_ e ɖɔ ajo xo ɔ nu mεsi ɔ bɔ e ɖɔ nuti hw...
11570,cela me suffit,START_ e kpé mì _END
9328,des chats,START_ awii ɖe lε _END
20092,tu vas trop vite,START_ a no ɖo wezun din _END
11681,brume d’harmattan,START_ akpitì _END
1067,chemin,START_ ali _END
12188,chef vodoun de dàn,START_ dannɔ̀ _END
10623,citerne souterraine,START_ lĭ _END


In [28]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((31500,), (3500,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [0]:
main_parameters = {
            
        "num_decoder_tokens" : num_decoder_tokens,
        "num_encoder_tokens" : num_encoder_tokens,
        "input_token_index" : input_token_index,
        "target_token_index" : target_token_index,
        "reverse_input_char_index": reverse_input_char_index,
        "reverse_target_char_index" : reverse_target_char_index,
        "latent_dim" : latent_dim,
        "max_length_tar" : max_length_tar,
        "max_length_src" : max_length_src,
        "input_words" : input_words,
        "target_words" : target_words
        }

with  open("main_parameters", 'wb') as file_:
    pickle.dump(main_parameters, file_)


In [33]:
# Encoder

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

num_encoder_tokens, latent_dim




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(12211, 50)

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
#from IPython.display import Image
#Image(retina=True, filename='train_model.png')

In [37]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
#batch_size = 64
epochs = 50

val_samples

3500

In [39]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
246/246 [==============================] - 49s 198ms/step - loss: 4.3108 - acc: 0.3180 - val_loss: 4.6777 - val_acc: 0.3119
Epoch 2/50
246/246 [==============================] - 48s 196ms/step - loss: 4.2567 - acc: 0.3237 - val_loss: 4.6623 - val_acc: 0.3179
Epoch 3/50
246/246 [==============================] - 48s 197ms/step - loss: 4.2065 - acc: 0.3300 - val_loss: 4.6459 - val_acc: 0.3197
Epoch 4/50
246/246 [==============================] - 49s 197ms/step - loss: 4.1459 - acc: 0.3357 - val_loss: 4.6211 - val_acc: 0.3234
Epoch 5/50
246/246 [==============================] - 48s 196ms/step - loss: 4.0946 - acc: 0.3416 - val_loss: 4.6013 - val_acc: 0.3270
Epoch 6/50
246/246 [==============================] - 48s 197ms/step - loss: 4.0516 - acc: 0.3471 - val_loss: 4.5922 - val_acc: 0.3293
Epoch 7/50
246/246 [==============================] - 48s 197ms/step - loss: 4.0096 - acc: 0.3526 - val_loss: 4.5746 - val_acc: 0.3329
Epoch 8/50
246/246 [==============================] - 4

### Always remember to save the weights

In [0]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [0]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
#Image(retina=True, filename='encoder_model.png')

In [0]:
#Image(retina=True, filename='decoder_model.png')

In [0]:

# Enregistrement des models dans des fichiers séparés
with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')

### Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [48]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: parle discrètement
Actual Marathi Translation:  đɔ̀ xó dɛ̆dɛ̆ 
Predicted Marathi Translation:  ye ɖo te kpɔ́n wɛ̀ 


In [49]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: cest pour moi
Actual Marathi Translation:  nyεtɔn wε 
Predicted Marathi Translation:  nù cé wɛ̀ 


In [50]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: a demain
Actual Marathi Translation:  mawu ni fɔn mi 
Predicted Marathi Translation:  e wa sɔ 


In [51]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: lenfant est une chose
Actual Marathi Translation:  vi nyi nu 
Predicted Marathi Translation:  vi ce ɖo 


In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: gibier à plume
Actual Marathi Translation:  xɛlàn 
Predicted Marathi Translation:  ɔ fesinnɔ 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: rapidité
Actual Marathi Translation:  wùyìyá wùyɛ̀yɛ́ 
Predicted Marathi Translation:  lε 


In [54]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: depuis il est reste fidèle à sa promesse
Actual Marathi Translation:  sin hwenεnu ɔ e nɔ gbeta tɔn ji 
Predicted Marathi Translation:  azan nabi e nɔ yi azɔmε ɖo gan mε sin ɔ mε 


In [55]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: etendre
Actual Marathi Translation:  tɛ́ 
Predicted Marathi Translation:  tɛ́ 


In [56]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il est libre
Actual Marathi Translation:  é ɖò é ɖésú’sí 
Predicted Marathi Translation:  é ɖò nù nŭ wɛ̀ 


In [57]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: gros
Actual Marathi Translation:  buwùn 
Predicted Marathi Translation:  ali 


In [58]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: un fils est né au village
Actual Marathi Translation:  vi ɖe nyi jiji ɖo gleta 
Predicted Marathi Translation:  vi ɔ ɖo nu kpɔ 


In [59]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: manchot
Actual Marathi Translation:  awagbonɔ̀ 
Predicted Marathi Translation:  sin 


In [60]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: devenir
Actual Marathi Translation:  zùn 
Predicted Marathi Translation:  hu 


In [61]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: quelque chose
Actual Marathi Translation:  nù tí 
Predicted Marathi Translation:  nɛ́ 


In [62]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il est réveillé depuis trois heures
Actual Marathi Translation:  e fɔn sin gan xixo atɔn ɖie 
Predicted Marathi Translation:  e ɖo xwe atɔn gan ɖokpo 


In [63]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: des ouvriers patients
Actual Marathi Translation:  azɔwatɔ sɔxwetɔ ɖe lɛ 
Predicted Marathi Translation:  e nɔ se myɔ kpεvi 


In [64]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: solide
Actual Marathi Translation:  gagogago 
Predicted Marathi Translation:  fún 


In [65]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: les moissonneurs sont allongés à l’ombre d’un arbre
Actual Marathi Translation:  jinukunyatɔ lε dlεn yeɖe ɖo yεtεn atin ɖe sa 
Predicted Marathi Translation:  folo lε ɖo ama da lɛ lɛ 


In [66]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: nom de garçon né le jeudi
Actual Marathi Translation:  kɔoví 
Predicted Marathi Translation:  ye 


In [67]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: a une autre fois
Actual Marathi Translation:  e sù hweɖénu 
Predicted Marathi Translation:  e ɖo zɔnlin ji wɛ 


In [68]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: avec insistance
Actual Marathi Translation:  tlítlí 
Predicted Marathi Translation:  kpodo kuku ɖokpo 


In [69]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: quel jour de marché estce aujourdhui
Actual Marathi Translation:  axitεgbe wε nyi egbe 
Predicted Marathi Translation:  xwe nabi mε nyi ɖo fi 


In [70]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tu gâtes trop ton enfant
Actual Marathi Translation:  a nɔ kɛ́ vì tówé dín 
Predicted Marathi Translation:  a nɔ gbɛ́ nukún tɔ́n lɛ́ bĭ 


In [71]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: combien dargent te faudratil
Actual Marathi Translation:  akwɛ́ nabí ká ná kpé wè 
Predicted Marathi Translation:  nabi a na mɔ ɖo fi elɔ 


In [72]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il obéit toujours à son instituteur
Actual Marathi Translation:  e nɔ se to nu nu mɛsi tɔn 
Predicted Marathi Translation:  e nɔ wazɔ nu ye gɔn 


In [73]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: maintenant et à lheure de notre mort
Actual Marathi Translation:  din kpodo kuhwenu mitɔn kpan 
Predicted Marathi Translation:  din kpodo kpɔ bo na mɔ mɔ mɛ ɔ 


In [74]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: si j’ai peur je me sauve
Actual Marathi Translation:  nu xɛsi ɖi mi ɔ na ɖido 
Predicted Marathi Translation:  nu un nɔ ɔ mi ɖɔ nuti 


In [75]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ils sont sérieusement au travail
Actual Marathi Translation:  yĕ ɖ’àzɔ̀ wú kútúkútú 
Predicted Marathi Translation:  yè nɔ́ mεsi dó ye 


In [76]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: certains oiseaux mangent du poisson cru
Actual Marathi Translation:  xɛvi ɖe lɛ nɔ ɖu hwevi mu 
Predicted Marathi Translation:  xεvi lε nɔ ɖu ganji 


In [77]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: cest trop blanc
Actual Marathi Translation:  e we faan 
Predicted Marathi Translation:  e we blo 


In [78]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: la chaise du professeur
Actual Marathi Translation:  azinkpo nukplɔnmɛtɔ tɔn 
Predicted Marathi Translation:  agban bu 


In [79]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: vipère à cornes
Actual Marathi Translation:  amakpwɛ̀ 
Predicted Marathi Translation:  ɔ azɔmɛ 


In [80]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: mon coeur tressaille de peur
Actual Marathi Translation:  hùn cé kɛ̀ 
Predicted Marathi Translation:  awu ce gblé 


In [81]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: prévision
Actual Marathi Translation:  nùnywɛ́ɖáyĭ 
Predicted Marathi Translation:  mε 


In [82]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ils ont bu de leau
Actual Marathi Translation:  ye nu sin 
Predicted Marathi Translation:  ye ɖo xɔ nu wε 


In [83]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: l’infirmière guérit les malades
Actual Marathi Translation:  doto ɔ ɖo azɔn gbɔ nu azizonnɔ lɛ wɛ 
Predicted Marathi Translation:  azinzɔnnɔ ɔ ɖo gbɔ nu nu azizonnɔ lε wε a 


In [84]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je veux seulement en fumer une
Actual Marathi Translation:  ɖokpo jε un nɔ jlo na nu 
Predicted Marathi Translation:  un nɔ wa fi a ɖo gan atɔɔn 


In [85]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: le dieu sème la richesse
Actual Marathi Translation:  dan nɔ do nu nu mε 
Predicted Marathi Translation:  nyɔnu ɖokpo ɖo na tɔn wε 


In [86]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il sappelle kossi
Actual Marathi Translation:  e nɔ ylɔ́ ɛ́ ɖɔ̀ kɔsí 
Predicted Marathi Translation:  e nɔ nyi zɔnlin ɖi mɛ à 


In [87]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: estce que je peux passer prendre ma voiture demain soir
Actual Marathi Translation:  un hεn ɔ na wa ze mɔto ce sɔ gbadanu a 
Predicted Marathi Translation:  un na ɔ sɔ ɖo azɔ enε ɔ a ɖe a ɖo na ko 


In [88]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: voici de la sauce de gombo
Actual Marathi Translation:  fevi nusunnu ɖie 
Predicted Marathi Translation:  kpodo kanlinnyitɔ 


In [89]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: libre
Actual Marathi Translation:  faà 
Predicted Marathi Translation:  ɖo 


In [90]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il est monté sur la montagne
Actual Marathi Translation:  e xa so ji 
Predicted Marathi Translation:  e ɖo so ji 


In [91]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: difficile
Actual Marathi Translation:  gɛ̀dɛ̀gɛ́dɛ́ 
Predicted Marathi Translation:  a 


In [92]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il faut courber la tête pour passer sous cette porte
Actual Marathi Translation:  e ɖo na do ta do bo na gbɔn hɔn enε ɔ gwlε 
Predicted Marathi Translation:  e na na lan bo na ɖu atin tɔn 


In [93]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je mange avec la bouche
Actual Marathi Translation:  un nɔ ɖu nu kpodo nu ce kpo 
Predicted Marathi Translation:  un nɔ do nu kpodo xwe atɔɔn 


In [94]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je viens de covè
Actual Marathi Translation:  un wa sín kɔ́vɛ̀ 
Predicted Marathi Translation:  un xɔ dó wè 


In [95]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: a demain
Actual Marathi Translation:  é sù ayì hɔ́n 
Predicted Marathi Translation:  e wa sɔ 


In [96]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: les marchandes étalent leurs marchandises sur leur éventaire
Actual Marathi Translation:  nùsátɔ́ lɛ́ nɔ́ tɛ́ agbàn ɖò kpancáɖá yètɔ́n jí 
Predicted Marathi Translation:  atin lε ɖo cuku yetɔn da kpo 


In [97]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: hé bien ils montrent un homme bien
Actual Marathi Translation:  hɛɛn e nɔ xlɛ sunnu ganji 
Predicted Marathi Translation:  nu ye nɔ da hu ganji 


In [98]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: estce quelle gagne beaucoup dargent
Actual Marathi Translation:  e nɔ ɖu akwε gege à 
Predicted Marathi Translation:  e ɖo te kpɔn xεsi nu wε bo ye 


In [99]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: corde
Actual Marathi Translation:  kan 
Predicted Marathi Translation:  


### Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [101]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: cest du café
Actual Marathi Translation:  kafe wɛ 
Predicted Marathi Translation:  sin hun ɔ 


In [102]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ornement
Actual Marathi Translation:  aɖawùn 
Predicted Marathi Translation:  ɖo 


In [103]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: la pluie
Actual Marathi Translation:  ji 
Predicted Marathi Translation:  jɛ́n 


In [104]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je tourne à droite
Actual Marathi Translation:  un ze ɖo alɔ ɖisi mε 
Predicted Marathi Translation:  un ɖò kɛ́n ɖokpó 


In [105]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: un voyou
Actual Marathi Translation:  aɖò 
Predicted Marathi Translation:  tɔ ɖe wε 


In [106]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je n’ai encore jamais entendu cette parole
Actual Marathi Translation:  un ko sè xó élɔ́ kpɔ́n ă 
Predicted Marathi Translation:  un mɔ mɔ̀ nù wú towe lɛ́ ná wá ă 


In [107]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: louche en fer
Actual Marathi Translation:  gànglwí 
Predicted Marathi Translation:  ɔ bi 


In [108]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: dessiner
Actual Marathi Translation:  đè 
Predicted Marathi Translation:  


In [109]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il parle sans cesse
Actual Marathi Translation:  é nɔ́ ɖɔ xó yuù 
Predicted Marathi Translation:  e ɖo xo ɖɔ wε a na 


In [110]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: atil changé
Actual Marathi Translation:  e húzú jijɔ à 
Predicted Marathi Translation:  e nyɔ́ nukún nú wè 


In [111]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: nous allons monter une tente pour y jouer avogan
Actual Marathi Translation:  mì nà dó ɖɛɖɛ dó xò avɔgán ná 
Predicted Marathi Translation:  mì ná yì dó xɔ́ntɔ́n wú 


In [112]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: nous approchons de portonovo
Actual Marathi Translation:  xɔgbonú kó yá 
Predicted Marathi Translation:  mi ɖo wema ɖokpo 


In [113]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: calcul
Actual Marathi Translation:  akɔ́wunta 
Predicted Marathi Translation:  nùsúnnú 


In [114]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: questce que kofi a dit
Actual Marathi Translation:  ani kofi ɖɔ 
Predicted Marathi Translation:  nε kofi ɖɔ 


In [115]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: imprimeur
Actual Marathi Translation:  wèmázíntɔ́ 
Predicted Marathi Translation:  kpodo kpo 


In [116]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tubercule de manioc
Actual Marathi Translation:  fɛnnyɛ́ 
Predicted Marathi Translation:  ɖo azinkpo 


In [117]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: qui vole avec effraction
Actual Marathi Translation:  xwéfotɔ́ 
Predicted Marathi Translation:  vi 


In [118]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: alors je suis allée à lécole en bus
Actual Marathi Translation:  bo un gbo do hun yi azomε 
Predicted Marathi Translation:  e un wa xwe ɔ a fɔn un ɖo akɔnkpikpan 


In [119]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: le marché s’anime
Actual Marathi Translation:  axi ɖo jijɛ wὲ 
Predicted Marathi Translation:  yi do nu finɛ 


In [120]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: car cest lui qui sauvera son peuple de ses péchés
Actual Marathi Translation:  ðó é na hwlɛn togun tɔn gán sín hwɛ tɔn lɛɛ mɛ 
Predicted Marathi Translation:  e nɔ yi wan nu azɔ tɔn na ɖo nu a e nyi wɛ nyi ye 


In [121]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: j’irai faire le marché du soir
Actual Marathi Translation:  un na yi xwlé agbònúxì 
Predicted Marathi Translation:  un na yi azɔ ce lɛ́ ná yí wè 


In [122]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il est devenu fou
Actual Marathi Translation:  é bɛ́ wlé 
Predicted Marathi Translation:  é kú dó 


In [123]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: et toi doù vienstu
Actual Marathi Translation:  hwε lo fitε a gosin 
Predicted Marathi Translation:  fítɛ́ a nɔ bló gbɔ̀n mɔ̆ 


In [124]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je me promène à lombre dun arbre
Actual Marathi Translation:  un ɖo yε bε ɖo atin ɔ gwlε 
Predicted Marathi Translation:  un ɖo wɔxuxu tɔn atin lε 


In [125]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: caché
Actual Marathi Translation:  bibɛ 
Predicted Marathi Translation:  mì 


In [126]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ca va couler en pure perte
Actual Marathi Translation:  e ná lúlú 
Predicted Marathi Translation:  e bɛ́ na yĕ 


In [127]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: moelle
Actual Marathi Translation:  fɔ̀n 
Predicted Marathi Translation:  kpodo kpo 


In [128]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: pâte levée
Actual Marathi Translation:  akánnújì 
Predicted Marathi Translation:  


In [129]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il m’imite
Actual Marathi Translation:  e nɔ́ vlɛ́ mì 
Predicted Marathi Translation:  e nɔ ɖi we à 


In [130]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: griot
Actual Marathi Translation:  hansinɔ̀ 
Predicted Marathi Translation:  


In [131]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: chagrin
Actual Marathi Translation:  nùvìvɛ́ 
Predicted Marathi Translation:  kpo 


In [132]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ils sont venus de làbas
Actual Marathi Translation:  dɔ̀n wɛ̀ yè sɔ́n 
Predicted Marathi Translation:  yè wá 


In [133]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: il est à moitié plein
Actual Marathi Translation:  é gɔ́ dó akpò 
Predicted Marathi Translation:  e ɖò ɖa dín 


In [134]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: pièce de centimes
Actual Marathi Translation:  tawungá 
Predicted Marathi Translation:  ɔ 


In [135]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: le toit de la maison
Actual Marathi Translation:  xwe ce sin xɔta 
Predicted Marathi Translation:  xwe ɔ́ ɖò xwe gbe gbe 


In [136]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: marteau
Actual Marathi Translation:  malitó 
Predicted Marathi Translation:  ɔ 


In [137]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: le travail doit être fini avant la fin de lannée
Actual Marathi Translation:  azɔ ɔ ɖo na nyi fifo cobɔ xwe o na fo 
Predicted Marathi Translation:  vi ɔ ɖo na nyi nu we ɖo fi ɔ sin kutɔnu 


In [138]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: que veuxtu acheter
Actual Marathi Translation:  etɛ xɔ gbe a ja 
Predicted Marathi Translation:  etε a xɔ nu 


In [139]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: promenade
Actual Marathi Translation:  sà 
Predicted Marathi Translation:  


In [140]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: mange un fruit
Actual Marathi Translation:  ɖu atinsinsɛn ɖokpo 
Predicted Marathi Translation:  ɖu atinsinsεn ɖokpo 


In [141]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: nous étudions le fon
Actual Marathi Translation:  mi ɖo fɔngbe kplɔn wɛ 
Predicted Marathi Translation:  mi ɖo fɔngbe kplɔn wε 


In [142]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ils lobligèrent à le faire
Actual Marathi Translation:  yè zɔ́n ɛ̀ gànnú ɖɔ̀ é ní bló 
Predicted Marathi Translation:  ye ɖo na wa 


In [143]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: elle nettoie son congélateur deux fois par an
Actual Marathi Translation:  e nɔ sunsun congélateur tɔn azɔn we ɖo xwe ɖokpo mε 
Predicted Marathi Translation:  e nɔ wazɔ bo ɖyɔ gege 


In [144]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ma vie est en sécurité
Actual Marathi Translation:  ayi ce j’ayĭ 
Predicted Marathi Translation:  nù cé wɛ̀ é nyí 


In [145]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: elle est en train de vendre sans contrôle
Actual Marathi Translation:  dùdúwɔgɔ tɔn sa w’ɛ ɖé 
Predicted Marathi Translation:  é ɖò nù kpódó ali kpán dín 


In [146]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: je veille sur lui
Actual Marathi Translation:  un tɔ́jú ì 
Predicted Marathi Translation:  un ɖo zɔnlin ɖi wɛ 


In [147]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: employé de bureau
Actual Marathi Translation:  akɔwé 
Predicted Marathi Translation:  e nɔ ɖi zɔnlin 


In [148]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: escalader
Actual Marathi Translation:  fán 
Predicted Marathi Translation:  e nɔ blo we 


In [149]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ces rivières sont poissonneuses
Actual Marathi Translation:  hwevitɔ wε nyi tɔ enε ɔ lε 
Predicted Marathi Translation:  bo nɔ ɖi xεsi nu hwan lε 
